### Import Packages

In [ ]:
import os
import json
import string
import numpy as np
import pandas as pd
from wordcloud import WordCloud
from matplotlib import pyplot as plt
from nltk.util import ngrams
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import PlaintextCorpusReader, stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram

### Create Document Corpus

In [ ]:
corpus_dir = "./Literature-original"
corpus = PlaintextCorpusReader(corpus_dir, ".*\.txt")
file_names = corpus.fileids()
file_names

### Corpus documents preprocessing

In [ ]:
documents = {}
for file_name in file_names:
    documents[file_name] = corpus.raw(file_name)
print(json.dumps(documents, indent=4, ensure_ascii=False))

In [ ]:
lengths = {}
for file_name in documents:
    lengths[file_name] = {
        "pre": len(word_tokenize(documents[file_name]))
    } 
    
print(json.dumps(lengths, indent=4, ensure_ascii=False))

In [ ]:
ps = PorterStemmer()

In [ ]:
for file_name in documents:
    documents[file_name] = documents[file_name].lower()
    documents[file_name] = "".join([char for char in documents[file_name] if char not in string.punctuation])
    documents[file_name] = "".join([char for char in documents[file_name] if not char.isdigit()])
    documents[file_name] = " ".join([ps.stem(word) for word in word_tokenize(documents[file_name])])
    documents[file_name] = " ".join([word for word in word_tokenize(documents[file_name]) if word not in list(stopwords.words('english'))])
print(json.dumps(documents, indent=4, ensure_ascii=False))


In [ ]:
for file_name in documents:
    lengths[file_name]['post'] = len(word_tokenize(documents[file_name]))
print(json.dumps(lengths, indent=4, ensure_ascii=False))

In [ ]:
lengths = pd.DataFrame.from_dict(lengths, 'index')

In [ ]:
lengths['diff'] = lengths['pre'] - lengths['post']
lengths['pct'] = lengths['diff'] / lengths['pre']
lengths

### Create frequency matrix

In [ ]:
docs = pd.DataFrame.from_dict(documents, orient='index')
docs.columns = ['content']
docs

In [ ]:
cv = CountVectorizer()
matrix_tf = cv.fit_transform(docs['content'])
matrix_tf 

In [ ]:
sparsity_tf = 1-(matrix_tf.getnnz()/(matrix_tf.shape[0]*matrix_tf.shape[1]))
sparsity_tf

In [ ]:
tv = TfidfVectorizer()
matrix_tfidf = tv.fit_transform(docs['content'])
matrix_tfidf

In [ ]:
sparsity_tfidf = 1-(matrix_tfidf.getnnz()/(matrix_tfidf.shape[0]*matrix_tfidf.shape[1]))
sparsity_tfidf

Directories for results

In [ ]:
if not os.path.exists("./wordclouds"):
    os.mkdir("./wordclouds")
if not os.path.exists("./topic_modelling"):
    os.mkdir("./topic_modelling")
if not os.path.exists("./topic_modelling/topics"):
    os.mkdir("./topic_modelling/topics")
if not os.path.exists("./topic_modelling/documents"):
    os.mkdir("./topic_modelling/documents")
if not os.path.exists("./clustering"):
    os.mkdir("./clustering")
if not os.path.exists("./ngrams"):
    os.mkdir("./ngrams")

Worldclouds

In [ ]:
wordcloud = WordCloud(
    background_color="white",
    max_words=5000,
    contour_width=3,
    contour_color='steelblue'
)

In [ ]:
for index, row in docs.iterrows():
    wordcloud.generate(row['content'])
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.title(index.replace(".txt", ""))
    plt.savefig("./wordclouds/{}".format(index.replace("txt", "png")))
    plt.close()



Topic modeling

In [ ]:
def plot_top_words(model, feature_names, n_top_words, title, size):
    colors = ['forestgreen', 'goldenrod', 'royalblue', 'slateblue', 'crimson',
              'peru', 'olivedrab', 'lightseagreen', 'cyan', 'lightcoral']
    fig, axes = plt.subplots(*size, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[-n_top_words:]
        top_features = feature_names[top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7, color=colors[topic_idx])
        ax.set_title(f"Topic {topic_idx + 1}", fontdict={"fontsize": 30})
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.savefig(f"./topic_modelling/topics/{title}.png")
    plt.close()

In [ ]:
def plot_documents(model, matrix, n_topics, title):
    colors = ['forestgreen', 'goldenrod', 'royalblue', 'slateblue', 'crimson',
            'peru', 'olivedrab', 'lightseagreen', 'cyan', 'lightcoral']
    docs_topics = pd.DataFrame(model.transform(matrix), columns=[f"Topic {x}" for x in range (n_topics)])
    docs_topics.index = [file_name.replace(".txt", "") for file_name in file_names]
    plt.figure(figsize=(7,4))
    left = [0] * len(docs_topics)
    for i, col in enumerate(docs_topics.columns):
        plt.barh(docs_topics.index, docs_topics[col], left=left, label=col, color=colors[i])
        left = [left[j]+docs_topics[col].iloc[j] for j in range(len(docs_topics))]
    plt.savefig(f"./topic_modelling/documents/{title}.png")
    plt.close()


In [ ]:
n_topics = 10
features_names = cv.get_feature_names_out()
n_top_words = 20
size = (2,5)

In [ ]:
lda = LatentDirichletAllocation(
    n_components=n_topics,
    max_iter=5,
    learning_method='online',
    learning_offset=50,
    random_state=0
)
lda.fit(matrix_tf)
plot_top_words(lda, features_names, n_top_words, "Topics in LDA", size )
plot_documents(lda, matrix_tf, n_topics, "Topics in LDA")

In [ ]:
nmf_fn = NMF(
    n_components=n_topics,
    random_state=1,
    alpha_H=0.00005,
    alpha_W=0.00005,
    l1_ratio=0.5
)
nmf_fn.fit(matrix_tfidf)
plot_top_words(nmf_fn, features_names, n_top_words, "Topics in LDA (FN)", size)
plot_documents(nmf_fn, matrix_tfidf, n_topics, "Topics in LDA (FN)")

In [ ]:
nmf_fn = NMF(
    n_components=n_topics,
    random_state=1,
    alpha_H=0.00005,
    alpha_W=0.00005,
    l1_ratio=0.5
)
nmf_fn.fit(matrix_tfidf)
plot_top_words(nmf_fn, features_names, n_top_words, "Topics in NMF (FN)", size)
plot_documents(nmf_fn, matrix_tfidf, n_topics, "Topics in NMF (FN)")

In [ ]:
nmf_kl = NMF(
    n_components=n_topics,
    random_state=1,
    alpha_H=0.00005,
    alpha_W=0.00005,
    l1_ratio=0.5,
    beta_loss='kullback-leibler',
    solver='mu',
    max_iter=1000
)
nmf_kl.fit(matrix_tfidf)
plot_top_words(nmf_kl, features_names, n_top_words, "Topics in NMF (KL)", size)
plot_documents(nmf_kl, matrix_tfidf, n_topics, "Topics in NMF (KL)")

Clustering

In [ ]:
def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
ed = euclidean_distances(matrix_tf, matrix_tf).flatten().reshape(matrix_tf.shape[0], matrix_tf.shape[0])

In [ ]:
clustering_ed_complete = AgglomerativeClustering(
    n_clusters=6,
    metric='precomputed',
    linkage='complete',
    compute_distances=True
)
clustering_ed_complete.fit(ed)
print(clustering_ed_complete.labels_)
plot_dendrogram(clustering_ed_complete, 
                labels=file_names,
                truncate_mode='level',
                orientation='right',
                )
plt.savefig('./clustering/ed_complete.png')
plt.close()

In [ ]:
cs = cosine_similarity(matrix_tfidf, matrix_tfidf).flatten().reshape(matrix_tfidf.shape[0], matrix_tfidf.shape[0])

In [ ]:
clustering_cs_ward = AgglomerativeClustering(
    n_clusters=6,
    metric='euclidean',
    linkage='ward',
    compute_distances=True
)
clustering_cs_ward.fit(cs)
print(clustering_cs_ward.labels_)
plot_dendrogram(clustering_cs_ward, 
                labels=file_names,
                truncate_mode='level',
                orientation='right',
                )
plt.savefig('./clustering/cs_ward.png')
plt.close()

N-grams

In [48]:
documents_tokenized = {}
for key in documents:
    documents_tokenized[key] = word_tokenize(documents[key], language='english')


In [55]:
tokens=5

for n in range(1,4):
    for title in documents_tokenized:
        n_gram = pd.Series(ngrams(documents_tokenized[title], n)).value_counts()
        n_gram[:tokens].plot.barh()
        plt.savefig(f'./ngrams/{n}_{title}.png')
        plt.close()
